# Day Pipeline Test

낮 파이프라인 전체 흐름을 테스트합니다.

```
Assets 로드
  ↓
WorldState 조회
  ↓
LockManager.check_unlocks() — 정보 해금
  ↓
DayController.process() — Tool 실행, turn += 1
  ↓
Delta 적용
  ↓
EndingChecker.check() — 엔딩 체크
  ↓
NarrativeLayer.render_day() 또는 render_ending()
```

## 0. Setup - 프로젝트 루트 탐색, 환경변수 로드

In [ ]:
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")

print(f"Project root: {ROOT}")

## 1. 시나리오 로드 + WorldState 준비

In [ ]:
from app.loader import ScenarioLoader
from app.schemas import WorldState, NPCState

loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = "coraline"  # 코렐라인 시나리오 사용
assets = loader.load(scenario_id)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {assets.get_all_npc_ids()}")
print(f"턴 제한: {assets.get_turn_limit()}")

# 테스트용 WorldState
world = WorldState(
    turn=2,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", trust=5, fear=0, suspicion=3),
        "button_father": NPCState(npc_id="button_father", trust=4, fear=0, suspicion=4),
        "button_daughter": NPCState(npc_id="button_daughter", trust=6, fear=0, suspicion=2),
    },
    inventory=[],
    vars={
        "humanity": 8,
        "total_suspicion": 3,
        "knife_touched": 0,
    },
)

print(f"\n초기 상태:")
print(f"  턴: {world.turn}")
print(f"  인간성: {world.vars['humanity']}")
print(f"  총 의심도: {world.vars['total_suspicion']}")

## 2. LockManager - 정보 해금 체크

In [ ]:
from app.lock_manager import get_lock_manager

lock_manager = get_lock_manager()
locks_data = assets.extras.get("locks", {})

print("=== LockManager 체크 ===")
lock_result = lock_manager.check_unlocks(world, locks_data)

print(f"새로 해금된 정보: {len(lock_result.newly_unlocked)}개")
for info in lock_result.newly_unlocked:
    print(f"  - [{info.info_id}] {info.info_title}")
    print(f"    {info.description}")
    print(f"    주입 대상: {', '.join(info.inject_to_npcs)}")

print(f"\n총 해금된 정보: {len(lock_result.all_unlocked)}개")

## 3. DayController - 낮 턴 실행

In [ ]:
from app.day_controller import get_day_controller

day_controller = get_day_controller()
user_input = "엄마에게 오늘 저녁은 뭐 먹을지 물어본다"

print(f"=== DayController 실행 ===")
print(f"유저 입력: {user_input}\n")

tool_result = day_controller.process(user_input, world, assets)

print(f"\n이벤트 설명:")
for desc in tool_result.event_description:
    print(f"  - {desc}")

print(f"\nstate_delta:")
print(f"  vars: {tool_result.state_delta.get('vars', {})}")
print(f"  npc_stats: {tool_result.state_delta.get('npc_stats', {})}")
print(f"  turn_increment: {tool_result.state_delta.get('turn_increment', 0)}")

## 4. Delta 적용 - WorldState 업데이트

In [ ]:
from app.state import get_world_state_manager

wsm = get_world_state_manager()
user_id = "test_user"

print("=== Delta 적용 ===")
print(f"적용 전 턴: {world.turn}")

# Delta 적용
world_after = wsm.apply_delta(user_id, scenario_id, tool_result.state_delta, assets)

print(f"적용 후 턴: {world_after.turn}")
print(f"\n변경된 상태:")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  총 의심도: {world.vars.get('total_suspicion')} → {world_after.vars.get('total_suspicion')}")

for npc_id in world.npcs:
    before = world.npcs[npc_id]
    after = world_after.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id) or {}
    npc_name = npc_data.get("name", npc_id)
    print(f"\n  [{npc_name}]")
    print(f"    trust: {before.trust} → {after.trust}")
    print(f"    suspicion: {before.suspicion} → {after.suspicion}")

## 5. EndingChecker - 엔딩 조건 체크

In [ ]:
from app.ending_checker import check_ending

print("=== EndingChecker ===")
ending_result = check_ending(world_after, assets)

if ending_result.reached:
    print(f"\n🎬 엔딩 도달!")
    print(f"  엔딩 ID: {ending_result.ending.ending_id}")
    print(f"  엔딩 이름: {ending_result.ending.name}")
    print(f"  epilogue_prompt: {ending_result.ending.epilogue_prompt[:100]}...")
    
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
else:
    print("엔딩 미도달 - 게임 계속")
    ending_info = None

## 6. NarrativeLayer - 나레이션 생성

In [ ]:
from app.narrative import get_narrative_layer

narrative = get_narrative_layer()

print("=== NarrativeLayer ===")

if ending_info:
    # 엔딩 나레이션
    print("\n[엔딩 나레이션 생성]")
    dialogue = narrative.render_ending(ending_info, world_after, assets)
else:
    # 일반 낮 나레이션
    print("\n[낮 나레이션 생성]")
    dialogue = narrative.render_day(
        tool_result.event_description,
        tool_result.state_delta,
        world_after,
        assets
    )

print("\n" + "=" * 60)
print(dialogue)
print("=" * 60)

## 7. 전체 파이프라인 결과 요약

In [ ]:
print("=" * 60)
print("Day Pipeline 테스트 완료")
print("=" * 60)
print(f"\n최종 상태:")
print(f"  턴: {world.turn} → {world_after.turn}")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  엔딩 도달: {'예 (' + ending_info['name'] + ')' if ending_info else '아니오'}")
print(f"\n해금된 정보: {len(lock_result.newly_unlocked)}개")
print(f"생성된 나레이션: {len(dialogue)}자")
print(f"\nDebug Info:")
print(f"  - LockManager: {len(lock_result.all_unlocked)}개 해금")
print(f"  - DayController: {len(tool_result.event_description)}개 이벤트")
print(f"  - EndingChecker: {'reached' if ending_result.reached else 'not reached'}")
print(f"  - NarrativeLayer: {'ending' if ending_info else 'day'} mode")